In [ ]:
#FINAL CODE
 
import mysql.connector
import math #FOR OTP
import smtplib#FOR OTP
import time
import pyperclip
import random
import os
from tabulate import tabulate
 
db = mysql.connector.connect(host="localhost",user="root",passwd="mysql",database="password")
print(db)
if db.is_connected():
    print("Successfully connected to the database")
 
mycursor = db.cursor()
 
def encode(password):    #crpytography
	encoded_chars= []
	for i in range(len(password)):
    	encoded_c = chr(ord(password[i]) - len(password))
        encoded_chars.append(encoded_c)
	encrypted_pass = "".join(encoded_chars)
	return encrypted_pass
    	
def decode(encrypted_pass):
	decoded_chars = []
	for i in range(len(encrypted_pass)):
    	decoded_c = chr(ord(encrypted_pass[i]) + len(encrypted_pass))
        decoded_chars.append(decoded_c)
	decoded_pass = "".join(decoded_chars)
	return decoded_pass
 
 
def menu():
	print(" ")
 
    print("WELCOME TO PASSWORD MANAGER")
	print("MAIN MENU:")
 
	print(" ")
 
    print("1.Enter new password")
	print("2.Edit password ")
    print("3.Access saved password")
    print("4.Forgot username/website")
    print("5.Delete password")
    print("6.Reset database & remove all the passwords.")
	print("7.Exit database")
	print("8.View database")
    print("9.Check password strength/suggestions")
	#frequency reminders
 
	print(" ")
 
	user_input = int(input("Please enter option from the above given menu: "))
 
	if user_input == 1:
    	add_pwd()
	elif user_input == 2:
    	edit_pwd()
	elif user_input == 3:
    	get_pwd()
	elif user_input == 4:
    	pwd_search()
	elif user_input == 5:
    	del_pwd()
	elif user_input == 6:
    	resetdb()
	elif user_input == 7:
    	app_exit()
	elif user_input == 8:
    	view()
	elif user_input == 9:
        passwordsuggest()
 
	else:
        print("ERROR: The option you have chosen is incorrect")
        print("Please choose the option from the given menu!!!")
    	print(" ")
 
    	print("Would you like to go back to menu?Yes/No?")
    	userInput = input("Enter option: ")
    	if userInput.lower()== "yes":
        	menu()
 
    	else:
            print("OK")
            	
 
def add_pwd():
	print("  ")
 
	print("Fill in the data below.")
	dates = input("Enter the date when password created/uploaded in yyyy/mm/dd format: ")
	websites = input("Enter Website: ")
	usernames = input("Enter Username or an Email_Id: ")
	passwords = input("Enter the password: ")
    passwordconfirm=input("Enter the password again (for confirmation):")
    encoded_pass=(encode(passwords))
 
	if passwords==passwordconfirm:
    	sql = "INSERT INTO passwords (Website, Username, Password, Date) VALUES (%s, %s, %s, %s)"
    	value = (websites, usernames, encoded_pass, dates)
 
        mycursor.execute(sql, value)
    	db.commit()
 
        print("Your password has been inserted into the database!!!")
 
    	print(" ")
 
    	print("Do you want to enter more passwords? ")
        print("Yes/No?")
 
	else:
        print("The passwords you have entered do not match")
        print("Would you like to try again?")
        print("Yes/No?")
    	
	userInput = input("Enter option: ")
	if userInput.lower()=="yes":
    	add_pwd()
	elif userInput.lower()=="no":
    	menu()
	else:
        print("ERROR ! wrong Input")
        print("Would you like to go back to menu?Yes/No?")
    	userInput = input("Enter choice: ")
  	  if userInput.lower()=="yes":
        	menu()
    	else:
            print("OK") 
    	
 
def edit_pwd():
	print("  ")
	print("Fill in the data below.")
	website = input("Enter Website: ")
	sql="select * from passwords where website=%s"
	val=(website,)
    mycursor.execute(sql,val)
    search=mycursor.fetchall()
	if bool(search)==False:
    	print("No passwords saved under this website")
        print("Would you like to try again?")
        print("Yes/No?")
	else:
    	usernames = input("Enter Username or an Email_Id: ")
    	for i in search:
        	if usernames in i:
                oldpass=input("Enter the old password:")
            	for i in search:
                    s=i[2]
                	p=decode(s)
                	if p==oldpass:
                        newpass=input("Enter the new password:")
                        newpassconfirm=input("Enter the new password again (for confirmation):")
                        if newpass==newpassconfirm:
                            sql="update passwords set password =%s where website=%s"
                            encoded_pass=encode(newpassconfirm)
                            val=(encoded_pass,website)
                            mycursor.execute(sql,val)
                            db.commit()
                            print("password changed successfully!")
                            print("Would you like to edit more passwords?")
                            print("Yes/No?")
                  	  else:
                            print("The passwords you have entered do not match")
                            print("Would you like to try again?")
                            print("Yes/No?")
                    else:
                        print("Incorrect old password!")
                        print("Do you want to edit more passwords? ")
                        print("Yes/No?")
        	else:
                print("The given username does not match any saved username")
                print("Would you like to try again?")
                print("Yes/No?")
        	
	userInput = input("Enter option: ")
	if userInput.lower()=="yes":
    	edit_pwd()
	elif userInput.lower()=="no":
    	menu()
	else:
 	   print("ERROR ! wrong Input")
        print("Would you like to go back to menu?Yes/No?")
    	userInput = input("Enter choice: ")
    	if userInput.lower()=="yes":
        	menu()
    	else:
            print("OK")
  
def get_pwd():
	print(" ")
	print("1) Search using  website name")
	print("2) Search using username")
 
	searchInput = int(input("Enter your option: "))
 
	if searchInput == 1:
    	web_search()
	elif searchInput == 2:
    	uname_search()
	else:
        print("ERROR:Wrong Input")
        print("Would you like to go back to menu?Yes/No?")
    	userInput = input("Enter choice: ")
    	if userInput.lower()=="yes":
        	menu()
    	else:
            print("OK")
 
 
def web_search():
    print("You have chosen to search the database using website name.")
 
    searchWebsite = input("Enter website: ")
 
    sql = "SELECT * FROM passwords WHERE Website = %s"
    val = (searchWebsite,)
    mycursor.execute(sql, val)
    search_result = mycursor.fetchall()
    l=[]
    for y in search_result:
        p,q,r,s=y
        print("the corresponding username is:",q)
        l.append(q)
    if len(l)>=2:
        print("you have 2 accounts for the same website")
        web=input("enter the account you want to continue with:")
        if web in l:
            s="Select * from passwords where Username=%s"
            val2=(web,)
            mycursor.execute(s,val2)
            search_result=mycursor.fetchall()
            for x in search_result:
                print("Copying the password to clipboard for 10 seconds")
                a,b,c,d=x
                s=decode(c)
                pyperclip.copy(s)
                time.sleep(10)
                pyperclip.copy('')
                print("Password erased from clipboard")
                print("Do you want to search database again?")
                print("Yes/No?")
                break
        else:
            print("sorry incorrect input!")
    elif len(l)==1:
        s="select * from passwords where Username=%s"
        val2=tuple(l)
        mycursor.execute(s,val2)
        search_result=mycursor.fetchall()
        for x in search_result:
            print("Copying the password to clipboard for 10 seconds")
            a,b,c,d=x
            s=decode(c)
            pyperclip.copy(s)
            time.sleep(10)
            pyperclip.copy('')
            print("Password erased from clipboard")
            print("Do you want to search database again?")
            print("Yes/No?")
            break
            
    else:
        print("Website not saved in database")
        print("Would you like to try again?")
        print("Yes/No?")
    print(" ")

    userInput = input("Enter choice: ")
    if userInput.lower()=="yes":
        get_pwd()
    elif userInput.lower()=="no":
        menu()
    else:
        print("ERROR:Wrong Input")
        print("Would you like to go back to menu?")
        print("Yes/No?")
        userInput = input("Enter choice: ")
        if userInput.lower()=="yes":
            menu()
        else:
            print("OK")
 
 
def uname_search():
    print("You have chosen to search the database using username.")
 
    searchUsername = input("Enter Username: ")
 
    sql = "SELECT * FROM passwords WHERE Username = %s"
    val = (searchUsername,)
    mycursor.execute(sql, val)
    search_result = mycursor.fetchall()
    l=[]
    for y in search_result:
        p,q,r,s=y
        print("the corresponding website is:",p)
        l.append(p)
    if len(l)>=2:
        print("you have 2 accounts for the same username")
        web=input("enter the account you want to continue with:")
        if web in l:
            s="Select * from passwords where Website=%s"
            val2=(web,)
            mycursor.execute(s,val2)
            search_result=mycursor.fetchall()
            for x in search_result:
                print("Copying the password to clipboard for 10 seconds")
                a,b,c,d=x
                s=decode(c)
                pyperclip.copy(s)
                time.sleep(10)
                pyperclip.copy('')
                print("Password erased from clipboard")
                print("Do you want to search database again?")
                print("Yes/No?")
                break
        else:
            print("sorry incorrect input!")
    elif len(l)==1:
        s="select * from passwords where Website=%s"
        val2=tuple(l)
        mycursor.execute(s,val2)
        search_result=mycursor.fetchall()
        for x in search_result:
            print("Copying the password to clipboard for 10 seconds")
            a,b,c,d=x
            s=decode(c)
            pyperclip.copy(s)
            time.sleep(10)
            pyperclip.copy('')
            print("Password erased from clipboard")
            print("Do you want to search database again?")
            print("Yes/No?")
            break
    
    else:
        print("Username not saved in database")
        print("Would you like to try again?")
        print("Yes/No?")
        
    print(" ")
    userInput = input("Enter choice: ")
    if userInput.lower()=="yes":
        get_pwd()
    elif userInput.lower()=="no":
        menu()
    else:
        print("ERROR:Wrong Input")
        print("Would you like to go back to menu?")
        print("Yes/No?")
        userInput = input("Enter choice: ")
        if userInput.lower()=="yes":
            menu()
        else:
            print("OK")
 
 
def pwd_search():
 
	searchPasswd = input("Enter Password: ")
    s=encode(searchPasswd)
 
	sql = "SELECT * FROM passwords WHERE Password = %s"
	val = (s,)
    mycursor.execute(sql, val)
	search_result = mycursor.fetchall()
	for x in search_result:
    	a,b,c,d=x
        print("Would you like to know the associated ","1)Website","2)Username","3)Date of Entry",sep='\n')
        choice=int(input("Enter choice:"))
    	if choice==1:
            print("Website:",a)
    	elif choice==2:
            print("Username:",b)
    	elif choice==3:
            print("Date of Entry:",d)
    	else:
  	      print("Wrong choice")
	print(" ")
 
	print("Do you want to search database again?")
 
	userInput = input( "Yes/No? ")
	if userInput.lower()=="yes":
    	pwd_search()
	elif userInput.lower()=="no":
    	menu()
	else:
        print("ERROR:Wrong Input")
        print("Would you like to go back to menu?")
        print("Yes/No?")
    	userInput = input("Enter choice: ")
    	if userInput.lower()=="yes":
        	menu()
    	else:
            print("OK")
 
def del_pwd():
	delete= input("Enter the website you want to delete the password from: ")
	sql = "delete from passwords where website = %s"
	val = (delete,)
    mycursor.execute(sql, val)
	db.commit()
	
	print("The data saved in the website you choose has been deleted from database.")
	print(" ")
	print("Do you want to delete again? ")
    print("Yes/No?")
	user_input = input("Enter choice: ")
	if user_input.lower()=="yes":
    	del_pwd()
	elif user_input.lower()=="no":
    	menu()
	else:
        print("ERROR:Wrong Input")
        print("Would you like to go back to menu?Yes/No?")
        userInput=input("Enter your choice:")
    	if userInput.lower()=="yes":
        	menu()
    	else:
            print("OK")
 
def resetdb():
	print("Enter 1 if you want to reset database.")
	print("Enter 2 if you want to go back to menu.")
 
	reset =int( input("Enter Option: "))
	if reset == 1:
        database_reset()
	elif reset == 2:
    	menu()
	else:
        print("ERROR:Wrong Input")
 
 
def database_reset():
    mycursor.execute("delete from passwords")
	db.commit()
 
	print("Your database has been completely erased.")
 
	print(" ")
 
def app_exit():
	print("Thank You for using password manager!")
	print("BYE SEE YOU SOON")
 
def view():
    mycursor.execute("select * from passwords")
	print('\n')
	headers=[[" WebsiteName","Username","Password","Date"]]
    print(tabulate(headers))
	for i in mycursor:
        values=fview(i)
        values[0][3]=i[3]
        print(tabulate(values))
	else:
        print("Database printed")
        choice=input("Do u want to continue to menu?")
    	if choice.lower()=="yes":
        	menu()
    	else:
            print("OK")
	
def fview(a):
    	j=a[2]
    	k=decode(j)
        finalview=[[a[0],a[1],k,' ']]
    	return finalview
    	
def passwordsuggest():
	print()
	print('Note: The password must be at least 6 characters, and no more than 12 characters long',
      	'Try to include special characters,combination of lowercase and uppercase characters',sep='\n')
	print()
 
 
	password = input ('Type your password:')
	recpass=password
	
	lower=False
	upper=False
	digit=False
	spch=False
	symbols = ['!', '#', '$', '%', '&', '*', '+','@']
    digit=['0','1','2','3','4','5','6','7','8','9']
 
	if len(password) >12:
    	print ('Password is too long.It must be between 6 and 12 characters')
    	print("Do u want to re-enter a password of appropriate length?")
        choice=input("Yes/No? \n Enter choice:")
    	if choice.lower()=='yes':
            print("Redirecting to enter password again")
            passwordsuggest()
    	else:
            print("OK.Moving ahead\n")
        	
	elif len(password)<6:
    	print ('Password is too short. It must be between 6 and 12 characters')
    	print("Do u want to re-enter a password of appropriate length?")
        choice=input("Yes/No? \n Enter choice:")
    	if choice.lower()=="yes":
            print("Redirecting to enter password again")
            passwordsuggest()
    	else:
            print("OK.Moving ahead\n")
        	
	elif len(password)>=6 and len(password)<= 12:
    	print ('Password length is perfect')
 
    	if password.islower()==True:
        	lower=True
        	print ('Password has only lowercase characters','Password strength:WEAK',sep='\n')
 
    	elif password.isupper()== True :
        	upper=True
        	print ('Password has only uppercase characters', 'Password strength:WEAK',sep='\n')
        	
    	elif password.isalnum()==False:
            print("Password has only special characters",'Password strength:WEAK',sep='\n')
        	spch=True
        	
    	elif password.isdigit()==True:
            print("Password has only digits",'Password strength:WEAK',sep='\n')
        	digit=True
        	
    	if lower==True or upper==True or spch==True or digit==True:
            print("******** Few suggestions to increase the strength of password are: *********")
            print("1.Passowrd should have both lower case and uppercase characters")
            print("2.Password should have atleast 1 special character")
            print("A few password recommendations:")
            sym=random.sample(symbols,1)
            sym2=random.sample(symbols,1)
        	a=''
        	b=''
        	c=''
        	for i in sym:
            	a=i
        	for i in sym2:
            	b=i
            dig=random.sample(digit,3)
        	for i in dig:
            	c+=i
            dig2=random.randint(0,10)
 	       print("1.",recpass+str(a)+str(c))
            print("2.",recpass.upper()+3*str(dig2))
            print("3.",recpass.capitalize()+str(dig2)+str(b))
            ans=ans=input("Do you want to go ahead and add one of these passwords to the database(yes/no)?")
        	if ans.lower()=='yes':
                print("1.Do you want to add a new record(website)?")
                print("2.Do you want to modify an existing record?")
                choice=int(input("Enter the choice:"))
            	if choice==1:
                    add_pwd()
            	elif choice==2:
                    edit_pwd()
        	else:
                print("Redirecting to menu")
            	menu()
    	if lower==False and upper==False and digit==False:
                print("Your password is strong!")

mycursor.execute('select * from masterpasswd;')
ls=mycursor.fetchall()
pas=input('Enter password')
for row in ls:
    ps=str(row[1])
    if ps==pas:
        print('logged in!')
        print("Welcome",row[0])
        check=otpverif()
        if check=="Verified":
            menu()
            break
else:
    print("INCORRECT PASSWORD")
    os.system("taskkill /F /im pythonw.exe")

 

